In [1]:
import pandas as pd 
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
from selenium.webdriver.chrome.options import Options
import cx_Oracle
import re

In [2]:
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))
driver.get("https://www.16personalities.com/ko/%EB%AC%B4%EB%A3%8C-%EC%84%B1%EA%B2%A9-%EC%9C%A0%ED%98%95-%EA%B2%80%EC%82%AC")
driver.implicitly_wait(3600)


button = driver.find_element(By.XPATH, '//*[@class="card__text"]')
ActionChains(driver).click(button).perform()
soup=BeautifulSoup(driver.page_source,'html.parser')
soup

[WDM] - Downloading: 100%|█████████████████████████████████████████████████████████| 6.30M/6.30M [00:00<00:00, 102MB/s]


<html lang="ko" style="--body-frozen-pad-rt: 0px;"><head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="" name="description"/>
<meta content="" name="author"/>
<meta content="IRocLTIXkDOJuw6g5wmkzrUeG7enurrF4KkW9Th3" name="csrf-token"/>
<meta content="v2.6.70" name="app-version"/>
<meta content="ko/type-description" name="app-route"/>
<meta content="0" name="app-bs-uid"/>
<meta content="" name="api-token"/>
<meta content="ko" name="app-locale"/>
<meta content="16Personalities" property="og:site_name"/>
<meta content="https://www.16personalities.com/ko/%ec%84%b1%ea%b2%a9%ec%9c%a0%ed%98%95-esfj" property="og:url"/>
<meta content="성격 유형: 집정관 (ESFJ) | 16Personalities" property="og:title"/>
<meta content="https://www.16personalities.com/static/images/social/esfj.png" property="og:image"/>
<title>성격 유형: 집정관 (ESFJ) | 16Personalities</title>
<link href="https://www.16personalit

##### MBTI결과만 뽑아내기 & 공백제거

In [3]:
table = soup.find('div', {'class' : ['results__type__code results__type__code--blue', 'results__type__code results__type__code--yellow',
                                    'results__type__code results__type__code--green', 'results__type__code results__type__code--purple']})
table

<div class="results__type__code results__type__code--blue" data-v-356ad83d="">
        ESFJ-T
      </div>

In [4]:
p = re.compile('[\n](.*?)[\n]')
table = p.findall(str(table))
print(table)

['        ESFJ-T']


In [5]:
table3 = pd.DataFrame({'URL':table})
table3

,URL
0,ESFJ-T


In [6]:
table3.URL = table3.URL.str.replace(' ', '')

In [7]:
table3.iloc[0]['URL']

'ESFJ-T'

##### INSERT

In [34]:
con = cx_Oracle.connect('mbti/1111@localhost:1521/xe')
cur = con.cursor()
sql_insert = '''
            insert into test(MBTI) 
            values(:MBTI)
            '''
MBTI = table3.iloc[0]['URL']

cur.execute(sql_insert, (MBTI,))


con.commit()
cur.close()
con.close()

##### UPDATE

In [35]:
con = cx_Oracle.connect('mbti/1111@localhost:1521/xe')
cur = con.cursor()
sql = "update movie set MBTI결과  = :1  WHERE 국문영화명 =:2"
MBTI = table3.iloc[0]['URL']

cur.execute(sql,(MBTI,'크레이지'))



con.commit()
cur.close()
con.close()